In [1]:
import requests
import urllib
from urllib.parse import quote
import pandas as pd
import time

In [2]:
# app 클라이언트 id 및 secret
client_id = '3KRFJUxrq1MnceEpl8__'
client_secret = 'sqx50WiesV'
headers = {'X-Naver-Client-Id':client_id, 
        'X-Naver-Client-Secret':client_secret}

url 형태 : "https://openapi.naver.com/v1/search/local.xml?query=%EC%A3%BC%EC%8B%9D&display=10&start=1&sort=random" <br>

In [3]:
# search_input = '경기도 용인시 수지구 풍덕천동 파스타집'

In [4]:
# # url 생성
# url_base="https://openapi.naver.com/v1/search/local.json?query="
# keyword = quote(search_input)

# url_2 = "&display="
# display = '5' # 한 번에 표시할 검색 결과 개수(기본값: 1, 최댓값: 5)

# url_3="$&start=" 
# start = '1' # 검색 시작 위치(기본값: 1, 최댓값: 1)

# url_4 = "&sort="
# # sort = 'random' # random: 정확도순으로 내림차순 정렬(기본값)
# sort = 'comment' # comment: 업체 및 기관에 대한 카페, 블로그의 리뷰 개수순으로 내림차순 정렬

# url = url_base + keyword + url_2 + display + url_3 + start + url_4 + sort

# result = requests.get(url, headers = headers).json()

In [5]:
# df = pd.DataFrame(result['items'])
# df['search_input'] = search_input
# df

In [6]:
name_df = pd.read_csv('geonggi_food_df.csv')
name_df = name_df[:10] # 샘플 n개
# name_df = name_df[10000:20000] # [a:b] a ~ b-1
name_df

,keyword
0,경기도 수원시 장안구 파장동 가정식
1,경기도 수원시 장안구 파장동 갈비
2,경기도 수원시 장안구 파장동 게요리
3,경기도 수원시 장안구 파장동 고기
4,경기도 수원시 장안구 파장동 고기뷔페
5,경기도 수원시 장안구 파장동 곰탕
6,경기도 수원시 장안구 파장동 곱창
7,경기도 수원시 장안구 파장동 과일쥬스
8,경기도 수원시 장안구 파장동 과자
9,경기도 수원시 장안구 파장동 국밥


In [7]:
def make_url(search_input):
    # url 생성
    url_base="https://openapi.naver.com/v1/search/local.json?query="
    keyword = quote(search_input)

    url_2 = "&display="
    display = '5' # 한 번에 표시할 검색 결과 개수(기본값: 1, 최댓값: 5)

    url_3="$&start=" 
    start = '1' # 검색 시작 위치(기본값: 1, 최댓값: 1)

    url_4 = "&sort="
    sort = 'random' # random: 정확도순으로 내림차순 정렬(기본값)
    # sort = 'comment' # comment: 업체 및 기관에 대한 카페, 블로그의 리뷰 개수순으로 내림차순 정렬

    url = url_base + keyword + url_2 + display + url_3 + start + url_4 + sort

    return url

In [8]:
def search_stores(search_input):
    
    url = make_url(search_input)

    while True:
        try:
            result = requests.get(url, headers = headers, timeout=15).json()
            time.sleep(0.1)

            df = pd.DataFrame(result['items'])
            df['search_input'] = search_input

            return df

        except requests.exceptions.Timeout:
                print('### requests.exceptions.Timeout ###')
                time.sleep(5)
                continue
    
        except TimeoutError:
                print('### TimeoutError ###')
                time.sleep(5)
                continue

        except ConnectionError:
                print('### ConnectionError ###')
                time.sleep(5)
                continue

        except KeyError:
                print('### KeyError ###')
                time.sleep(5)
                continue
    

In [9]:
# 빈 데이터프레임 생성
col = ['title', 'link', 'category', 'description', 'telephone', 'address', 'roadAddress', 'mapx', 'mapy', 'search_input']
result_df = pd.DataFrame(columns=col)
result_df

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input


In [10]:
cnt = 0
for kw in name_df['keyword']:
    print('cnt:', cnt)
    print('*'*10, kw, '*'*10)
    add_df = search_stores(kw)
    result_df = pd.concat([result_df, add_df])
    cnt += 1
result_df = result_df.reset_index(drop=True)

cnt: 0
********** 경기도 수원시 장안구 파장동 가정식 **********
cnt: 1
********** 경기도 수원시 장안구 파장동 갈비 **********
cnt: 2
********** 경기도 수원시 장안구 파장동 게요리 **********
cnt: 3
********** 경기도 수원시 장안구 파장동 고기 **********
cnt: 4
********** 경기도 수원시 장안구 파장동 고기뷔페 **********
cnt: 5
********** 경기도 수원시 장안구 파장동 곰탕 **********
cnt: 6
********** 경기도 수원시 장안구 파장동 곱창 **********
cnt: 7
********** 경기도 수원시 장안구 파장동 과일쥬스 **********
cnt: 8
********** 경기도 수원시 장안구 파장동 과자 **********
cnt: 9
********** 경기도 수원시 장안구 파장동 국밥 **********


In [11]:
len(result_df)

47

In [12]:
result_df.head()

,title,link,category,description,telephone,address,roadAddress,mapx,mapy,search_input
0,토담골,,"한식>백반,가정식",,,경기도 수원시 장안구 파장동 578-18 1층,경기도 수원시 장안구 경수대로1073번길 24 1층,311034,523384,경기도 수원시 장안구 파장동 가정식
1,현정이네식당,,"한식>백반,가정식",,,경기도 수원시 장안구 파장동 595-16,경기도 수원시 장안구 경수대로1031번길 8 한국일보,311442,523255,경기도 수원시 장안구 파장동 가정식
2,이목식당,,"한식>백반,가정식",,,경기도 수원시 장안구 이목동 210-10,경기도 수원시 장안구 장안로 433,309873,524501,경기도 수원시 장안구 파장동 가정식
3,엄마밥상,,"한식>백반,가정식",,,경기도 수원시 장안구 파장동 267-1,경기도 수원시 장안구 경수대로1078번길 17-1 경진만물전자,311107,523601,경기도 수원시 장안구 파장동 가정식
4,행복한밥상,,"한식>백반,가정식",,,경기도 수원시 장안구 조원동 774-2,경기도 수원시 장안구 송정로 164,312842,522231,경기도 수원시 장안구 파장동 가정식


In [13]:
# 중복 행 확인
result_df.duplicated(['mapx','mapy']).sum()

4

In [14]:
print('중복제거 이전 길이:', len(result_df))

result_df_d = result_df.drop_duplicates(['mapx','mapy']) # 중복제거
result_df_d = result_df_d.reset_index(drop=True)
print('중복제거 이후 길이:', len(result_df_d))

중복제거 이전 길이: 47
중복제거 이후 길이: 43


In [15]:
result_df_d.to_csv('gg_search_sample.csv',encoding='utf-8-sig',index=False)